## Configuration

### Proxy Role
AWS Role - 'arn:aws:iam::946429944765:role/irsa-user-sa-proxy-role'
Trust Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::946429944765:oidc-provider/oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260BA46DDF"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringLike": {
                    "oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260BA46DDF:aud": "sts.amazonaws.com",
                    "oidc.eks.us-west-2.amazonaws.com/id/4BFF6850498B7D7ED319EF260BA46DDF:sub": [
                        "*:domino-compute:integration-test",
                        "*:domino-compute:john-doe",
                        "*:domino-compute:jane-doe"
                    ]
                }
            }
        }
    ]
}
```
Permission Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sts:AssumeRole",
            "Resource": "arn:aws:iam::946429944765:role/irsa-user-sa-asset-role"
        }
    ]
}
```

### Asset Role
AWS Role - 'arn:aws:iam::946429944765:role/irsa-user-sa-asset-role'
Trust Policy
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::946429944765:role/irsa-user-sa-proxy-role"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
```
Permission Policy - Undefined for this demo. But can be anything the admin allows, say listing of a S3 bucket.


In [1]:
import boto3
import os

In [2]:
os.environ['AWS_ROLE_ARN']='arn:aws:iam::946429944765:role/irsa-user-sa-proxy-role'

In [3]:
sts = boto3.client('sts')
sts.get_caller_identity()

{'UserId': 'AROA5YW464O6ZWY7SOMFY:botocore-session-1707141640',
 'Account': '946429944765',
 'Arn': 'arn:aws:sts::946429944765:assumed-role/irsa-user-sa-proxy-role/botocore-session-1707141640',
 'ResponseMetadata': {'RequestId': '99f4beba-3896-4564-915c-9a4e86f5b5a3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '99f4beba-3896-4564-915c-9a4e86f5b5a3',
   'content-type': 'text/xml',
   'content-length': '484',
   'date': 'Mon, 05 Feb 2024 14:00:39 GMT'},
  'RetryAttempts': 0}}

In [7]:
response = sts.assume_role(
    RoleArn="arn:aws:iam::946429944765:role/irsa-user-sa-asset-role",
    RoleSessionName="irsa-user-sa-asset-role"
)
print(response['AssumedRoleUser'])

{'AssumedRoleId': 'AROA5YW464O6ZQVCRZJM4:irsa-user-sa-asset-role', 'Arn': 'arn:aws:sts::946429944765:assumed-role/irsa-user-sa-asset-role/irsa-user-sa-asset-role'}
